# Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference


Welcome to this getting started guide, we will use the new Hugging Face Inference DLCs and Amazon SageMaker Python SDK to deploy a transformer model for real-time inference. 
In this example we are going to deploy a trained Hugging Face Transformer model on to SageMaker for inference.

## API - [SageMaker Hugging Face Inference Toolkit](https://github.com/aws/sagemaker-huggingface-inference-toolkit)


Using the `transformers pipelines`, we designed an API, which makes it easy for you to benefit from all `pipelines` features. The API is oriented at the API of the [🤗  Accelerated Inference API](https://api-inference.huggingface.co/docs/python/html/detailed_parameters.html), meaning your inputs need to be defined in the `inputs` key and if you want additional supported `pipelines` parameters you can add them in the `parameters` key. Below you can find examples for requests. 

**text-classification request body**
```python
{
	"inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}
```
**question-answering request body**
```python
{
	"inputs": {
		"question": "What is used for inference?",
		"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}
```
**zero-shot classification request body**
```python
{
	"inputs": "Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
	"parameters": {
		"candidate_labels": [
			"refund",
			"legal",
			"faq"
		]
	}
}
```

In [ ]:
!pip install "sagemaker>=2.48.0" --upgrade
# !pip install torch -q
!pip install transformers -q

In [ ]:
import sagemaker
sagemaker.__version__

#### **Note: Restart the notebook after installing the above packages.**

## Deploy a Hugging Face Transformer model from S3 to SageMaker for inference


There are two ways on how you can deploy you SageMaker trained Hugging Face model from S3. You can either deploy it after your training is finished or you can deploy it later using the `model_data` pointing to you saved model on s3.

### Deploy the model directly after training

If you deploy you model directly after training you need to make sure that all required files are saved in your training script, including the Tokenizer and the Model. 
```python
from sagemaker.huggingface import HuggingFace

############ pseudo code start ############

# create HuggingFace estimator for running training
huggingface_estimator = HuggingFace(....)

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(...)

############ pseudo code end ############

# deploy model to SageMaker Inference
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

# example request, you always need to define "inputs"
data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

# request
predictor.predict(data)

```



### Download Hugging Face Pretrained Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
MODEL = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model.save_pretrained('my_model')
tokenizer.save_pretrained('my_model')

### Package the saved model to tar.gz formant

In [7]:
!cd my_model && tar zcvf model.tar.gz * 
!mv my_model/model.tar.gz ./model.tar.gz

a config.json
a pytorch_model.bin
a special_tokens_map.json
a tokenizer.json
a tokenizer_config.json
a vocab.txt


### Upload Pretrained Model to S3

In [ ]:
import sagemaker
from sagemaker.s3 import S3Uploader,s3_path_join

# get the s3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()
# uploads a given file to S3.
upload_path = s3_path_join("s3://",sagemaker_session_bucket,"lab1_model")
model_uri = S3Uploader.upload('model.tar.gz',upload_path)
print(f"Uploading Model to {upload_path}")
print(f" model.tar.gz uploaded to {model_uri}")

### Deploy the model using `model_data`

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=model_uri,  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.4xlarge"
)

**Architecture**

The [Hugging Face Inference Toolkit for SageMaker](https://github.com/aws/sagemaker-huggingface-inference-toolkit) is an open-source library for serving Hugging Face transformer models on SageMaker. It utilizes the SageMaker Inference Toolkit for starting up the model server, which is responsible for handling inference requests. The SageMaker Inference Toolkit uses [Multi Model Server (MMS)](https://github.com/awslabs/multi-model-server) for serving ML models. It bootstraps MMS with a configuration and settings that make it compatible with SageMaker and allow you to adjust important performance parameters, such as the number of workers per model, depending on the needs of your scenario.

![](../../imgs/hf-inference-toolkit.png)

**Deploying a model using SageMaker hosting services is a three-step process:**

1. **Create a model in SageMaker** —By creating a model, you tell SageMaker where it can find the model components. 
2. **Create an endpoint configuration for an HTTPS endpoint** —You specify the name of one or more models in production variants and the ML compute instances that you want SageMaker to launch to host each production variant.
3. **Create an HTTPS endpoint** —Provide the endpoint configuration to SageMaker. The service launches the ML compute instances and deploys the model or models as specified in the configuration

![](../../imgs/sm-endpoint.png)


In [ ]:
# example request, you always need to define "inputs"
data = {
   "inputs": "The new Hugging Face SageMaker DLC makes it super easy to deploy models in production. I love it!"
}

# request
predictor.predict(data)

## Model Monitoring



In [ ]:
for i in range(1000):
    predictor.predict(data)

## Clean up

In [ ]:
# delete endpoint
predictor.delete_endpoint()